# Agent

In [ ]:

import json
from fastapi.responses import StreamingResponse
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langchain.tools import tool
from src.core.utils.logger import get_logger

logger = get_logger(__name__)

@tool
def get_weather(location: str) -> str:
    """Get the weather for a location"""
    # Fake weather data
    return f"The weather in {location} is sunny and 25°C"

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search_web(query: str) -> str:
    """Search the web for information"""
    return f"Search results for '{query}': Found 10 results about {query}"

tools = [get_weather, calculate, search_web]



In [ ]:

from langchain.agents.middleware import wrap_tool_call

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

In [43]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gpt-5-nano",
    max_completion_tokens=1048,
    timeout=30,
)
agent = create_agent(model, tools=tools,    
                     middleware=[handle_tool_errors],
                     system_prompt="You are a helpful assistant. Be concise and accurate.")


# Response output

- AIMessage
- AIMessageChunk (stream)
  - Source
  - Suggestion
- ToolMessage

- UserMessage

## Output schema



# Experiments with models

## get text content

In [44]:
# normal text content, no streaming
model_with_tools = model.bind_tools(tools)
result = model_with_tools.invoke("What is the weather in New York?")

In [46]:
from pprint import pprint
# from langchain_core.messages import message_to_dict
# message = message_to_dict(result)

message = result.model_dump()

pprint(message)

{'additional_kwargs': {'refusal': None},
 'content': '',
 'id': 'lc_run--019b17ec-4106-7ab1-9aa6-b2a3ca1f7f4b-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': 'chatcmpl-CmK4E0Pc8FutNB5XPtk8XWNiiJrgk',
                       'logprobs': None,
                       'model_name': 'gpt-5-nano-2025-08-07',
                       'model_provider': 'openai',
                       'service_tier': 'default',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 152,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 128,
                                                                     'rejected_prediction_tokens': 0},
            

In [47]:
response = model.invoke("Why do parrots have colorful feathers?")
reasoning_steps = [b for b in response.content_blocks if b["type"] == "reasoning"]

In [48]:
response.model_dump()

{'content': '',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 1048,
   'prompt_tokens': 14,
   'total_tokens': 1062,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 1048,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_provider': 'openai',
  'model_name': 'gpt-5-nano-2025-08-07',
  'system_fingerprint': None,
  'id': 'chatcmpl-CmK4e8eADr6pJFzylDyPn19SnuLk5',
  'service_tier': 'default',
  'finish_reason': 'length',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019b17ec-a79b-7bc0-802f-d96a666a2740-0',
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 14,
  'output_tokens': 1048,
  'total_tokens': 1062,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 1048}}}

In [27]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])  

response = model_with_tools.invoke("What's the weather like in Boston?")
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

Tool: get_weather
Args: {'location': 'Boston'}


# Experiments with agent

## output schema
content,
reasoning tokens,
tool calls,
tool fails
source url
source inline
usage data

```python
class UIMessageChunk:
  

```

In [28]:
agent_response = agent.invoke({"messages": [HumanMessage(
  content="""
            Question 1: What is the weather in New York? 
            Question 2: Use reasoning to answer this. Give me the smallest positive integer that has exactly 10 divisors.
            Show every deduction you make and justify why your answer is the smallest.""")]})

In [29]:
agent_response.get("messages")

[HumanMessage(content='\n            Question 1: What is the weather in New York? \n            Question 2: Use reasoning to answer this. Give me the smallest positive integer that has exactly 10 divisors.\n            Show every deduction you make and justify why your answer is the smallest.', additional_kwargs={}, response_metadata={}, id='959818b7-ff42-4695-8ee0-7e9f449efaef'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 178, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_d0c93c37b1', 'id': 'chatcmpl-CmJn9CWoN6kZJGWGJgVr0AJLmLSFG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--01

In [30]:
agent_response_stream =  agent.stream({"messages": [HumanMessage(content="What is the weather in New York?, Sơn đến sớm hơn Hương, nhưng muộn hơn Ngọc. Ai là người đến sớm nhất?")]})

for chunk in agent_response_stream:
    print(chunk)


{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 154, 'total_tokens': 204, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_d0c93c37b1', 'id': 'chatcmpl-CmJnI022bi5uFGrGjGspxmdz1C6fH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b17dc-3e86-7a30-9a3f-7618ecc16324-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'New York'}, 'id': 'call_C1t2xdfALgEEnRGczw875X44', 'type': 'tool_call'}, {'name': 'calculate', 'args': {'expression': 'Ngọc < Sơn < Hương'}, 'id': 'call_rNJDDisMk0h1jh1gsguW0G60', 'type': 'tool_call'}], usage_metadata={'input_tokens': 154, 'output_tokens': 50, 'to

In [49]:
agent_response_stream = agent.stream({"messages": [HumanMessage(
    content="What is the weather in New York?, What is 3^3?")]})

for chunk in agent_response_stream:
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content_blocks: {data['messages'][-1].content_blocks}")
        print(f"content: {chunk}")


step: model
content_blocks: [{'type': 'tool_call', 'name': 'get_weather', 'args': {'location': 'New York'}, 'id': 'call_wE1HY8llie8Rj7LkiumVI5j6'}, {'type': 'tool_call', 'name': 'calculate', 'args': {'expression': '3^3'}, 'id': 'call_JIJx2jAIXoVcIHzdTaJ1e4sz'}]
content: {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 311, 'prompt_tokens': 222, 'total_tokens': 533, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CmK5BLPaYs5SUkVUHX3l8kkuymxsT', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b17ed-2641-7603-aabb-92bb596e524e-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'New

## Clean the code

- AIMessage
- AIMessageChunk (stream)
  - Source
  - Suggestion
- ToolMessage

- UserMessage

# Experiments with multi-agents